## PyCity Schools Analysis
### - 
### - 
### - 

In [393]:
import pandas as pd
import numpy as np

df_students = pd.read_csv("students_complete.csv")
df_schools = pd.read_csv("schools_complete.csv")

# total math and reading passing grade added to school df
ayy = pd.DataFrame(df_students['school'].loc[df_students['math_score'] >= 70].value_counts())
lmao = pd.DataFrame(df_students['school'].loc[df_students['reading_score'] >= 70].value_counts())
ayy.reset_index(inplace=True)
lmao.reset_index(inplace=True)
ayylmao = pd.merge(ayy,lmao,on='index')
ayylmao.columns = ["name", "math_pass","reading_pass"]
ayylmao
df_schools = pd.merge(df_schools,ayylmao,on='name')

### Distric Summary

In [288]:
math_path = df_students["math_score"].loc[df_students["math_score"] >= 70].count()/df_schools['size'].sum() * 100
read_path = df_students["reading_score"].loc[df_students["reading_score"] >= 70].count()/df_schools['size'].sum() * 100
dist_sum = pd.DataFrame(
        {"Total Schools": df_schools['name'].count(),
         "Total Students": "{:,}".format(df_schools['size'].sum()),
         "Total Budget": "${:,.2f}".format(df_schools['budget'].sum()),
         "Average Math Score": "{:.2f}".format(df_students['math_score'].mean()),
         "Average Reading Score": "{:.2f}".format(df_students ['reading_score'].mean()),
         "% Passing Math": "{:.2f}%".format(math_path),
         "% Passing Reading": "{:.2f}%".format(read_path),
         "% Overall Passing Rate": "{:.2f}%".format(np.mean([math_path,read_path]))}, index = [0])
dist_sum = dist_sum[["Total Schools", "Total Students", "Total Budget", "Average Math Score",
                     "Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing Rate"]]
dist_sum

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98%,85.81%,80.39%


### School Summary

In [397]:
sch_sum = df_schools.loc[:,['name','type','size','budget']]
sch_sum.columns = ['name','School Type','Total Students','Total School Budget']
sch_sum['Per Student Budget'] = df_schools['budget']/df_schools['size']
sch_sum['Average Math Score'] = df_students['math_score'].mean()
sch_sum['Average Reading Score'] = df_students['reading_score'].mean()
sch_sum['% Passing Math'] = df_schools['math_pass']/df_schools['size']*100
sch_sum['% Passing Reading'] = df_schools['reading_pass']/df_schools['size']*100
sch_sum['% Overall Passing Rate'] = (sch_sum['% Passing Math']+sch_sum['% Passing Reading'])/2
sch_sum = sch_sum.set_index('name')
sch_sum.index.name = None
sch_sum.sort_index(inplace=True)
sch_sum_final = sch_sum.style.format({"Total School Budget": "${:,.2f}", "Per Student Budget":"${:,.2f}", "Average Math Score":"{:.2f}", 
                      "Average Reading Score":"{:.2f}", "% Passing Math":"{:.2f}%", 
                      "% Passing Reading":"{:.2f}%", "% Overall Passing Rate":"{:.2f}%"})
sch_sum_final

## Top Performing Schools (By Passing Rate)

In [290]:
sch_sum.sort_values(['% Overall Passing Rate'],ascending = False).head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Cabrera High School,Charter,1858,1081356,582.0,78.985371,81.87784,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,1043130,638.0,78.985371,81.87784,93.272171,97.308869,95.290520
Pena High School,Charter,962,585858,609.0,78.985371,81.87784,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,917500,625.0,78.985371,81.87784,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,1319574,578.0,78.985371,81.87784,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

In [291]:
sch_sum.sort_values(['% Overall Passing Rate'],ascending = True).head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Rodriguez High School,District,3999,2547363,637.0,78.985371,81.87784,66.366592,80.220055,73.293323
Figueroa High School,District,2949,1884411,639.0,78.985371,81.87784,65.988471,80.739234,73.363852
Huang High School,District,2917,1910635,655.0,78.985371,81.87784,65.683922,81.316421,73.500171
Johnson High School,District,4761,3094650,650.0,78.985371,81.87784,66.057551,81.222432,73.639992
Ford High School,District,2739,1763916,644.0,78.985371,81.87784,68.309602,79.299014,73.804308


## Math Scores by Grade

In [372]:
math_temp = df_students.loc[:,['grade','school','math_score']]
math_gr_9 = math_temp.loc[temp['grade'] == "9th"].groupby(['school']).mean()
math_gr_9.reset_index(level=0, inplace=True)
math_gr_10 = math_temp.loc[temp['grade'] == "10th"].groupby(['school']).mean()
math_gr_10.reset_index(level=0, inplace=True)
math_gr_11 = math_temp.loc[temp['grade'] == "11th"].groupby(['school']).mean()
math_gr_11.reset_index(level=0, inplace=True)
math_gr_12 = math_temp.loc[temp['grade'] == "12th"].groupby(['school']).mean()
math_gr_12.reset_index(level=0, inplace=True)
math_gr = math_gr_9.merge(math_gr_10,on = 'school').merge(math_gr_11,on = 'school').merge(math_gr_12,on='school')
math_gr.columns = ['school','9th','10th','11th','12th']
math_gr = math_gr.set_index('school')
del math_gr.index.name
math_gr

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade

In [373]:
read_temp = df_students.loc[:,['grade','school','reading_score']]
read_gr_9 = read_temp.loc[temp['grade'] == "9th"].groupby(['school']).mean()
read_gr_9.reset_index(level=0, inplace=True)
read_gr_10 = read_temp.loc[temp['grade'] == "10th"].groupby(['school']).mean()
read_gr_10.reset_index(level=0, inplace=True)
read_gr_11 = read_temp.loc[temp['grade'] == "11th"].groupby(['school']).mean()
read_gr_11.reset_index(level=0, inplace=True)
read_gr_12 = read_temp.loc[temp['grade'] == "12th"].groupby(['school']).mean()
read_gr_12.reset_index(level=0, inplace=True)
read_gr = read_gr_9.merge(read_gr_10,on = 'school').merge(read_gr_11,on = 'school').merge(read_gr_12,on='school')
read_gr.columns = ['school','9th','10th','11th','12th']
read_gr = read_gr.set_index('school')
del read_gr.index.name
read_gr

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

In [401]:
sss_temp = sch_sum.loc[:,['Per Student Budget','Average Math Score','Average Reading Score',
                          '% Passing Math','% Passing Reading','% Overall Passing Rate']]
sss_temp = sss_temp.set_index('Per Student Budget')


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Per Student Budget,,,,,
628.0,78.985371,81.87784,66.680064,81.933280,74.306672
582.0,78.985371,81.87784,94.133477,97.039828,95.586652
639.0,78.985371,81.87784,65.988471,80.739234,73.363852
644.0,78.985371,81.87784,68.309602,79.299014,73.804308
625.0,78.985371,81.87784,93.392371,97.138965,95.265668
652.0,78.985371,81.87784,66.752967,80.862999,73.807983
581.0,78.985371,81.87784,92.505855,96.252927,94.379391
655.0,78.985371,81.87784,65.683922,81.316421,73.500171
650.0,78.985371,81.87784,66.057551,81.222432,73.639992


## Scores by School Size

In [405]:
ssz_temp = sch_sum.loc[:,['Total Students','Average Math Score','Average Reading Score',
                          '% Passing Math','% Passing Reading','% Overall Passing Rate']]
ssz_temp = ssz_temp.set_index('Total Students')
ssz_temp

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Total Students,,,,,
4976,78.985371,81.87784,66.680064,81.933280,74.306672
1858,78.985371,81.87784,94.133477,97.039828,95.586652
2949,78.985371,81.87784,65.988471,80.739234,73.363852
2739,78.985371,81.87784,68.309602,79.299014,73.804308
1468,78.985371,81.87784,93.392371,97.138965,95.265668
4635,78.985371,81.87784,66.752967,80.862999,73.807983
427,78.985371,81.87784,92.505855,96.252927,94.379391
2917,78.985371,81.87784,65.683922,81.316421,73.500171
4761,78.985371,81.87784,66.057551,81.222432,73.639992


## Scores by School Type

In [407]:
sst_temp = sch_sum.loc[:,['School Type','Average Math Score','Average Reading Score',
                          '% Passing Math','% Passing Reading','% Overall Passing Rate']]
sst_temp

,School Type,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Bailey High School,District,78.985371,81.87784,66.680064,81.933280,74.306672
Cabrera High School,Charter,78.985371,81.87784,94.133477,97.039828,95.586652
Figueroa High School,District,78.985371,81.87784,65.988471,80.739234,73.363852
Ford High School,District,78.985371,81.87784,68.309602,79.299014,73.804308
Griffin High School,Charter,78.985371,81.87784,93.392371,97.138965,95.265668
Hernandez High School,District,78.985371,81.87784,66.752967,80.862999,73.807983
Holden High School,Charter,78.985371,81.87784,92.505855,96.252927,94.379391
Huang High School,District,78.985371,81.87784,65.683922,81.316421,73.500171
Johnson High School,District,78.985371,81.87784,66.057551,81.222432,73.639992
Pena High School,Charter,78.985371,81.87784,94.594595,95.945946,95.270270
